In [1]:
import typing as t
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
tqdm.pandas()

In [3]:
DATA_DIR = Path('/root/data')
DATASET_DIR = DATA_DIR / 'datasets'
OOF_DIR = DATA_DIR / 'oof'
FP_DATASET_DIR = DATASET_DIR / 'feedback-prize-effectiveness'

In [4]:
train_df = pd.read_csv(FP_DATASET_DIR / 'train_ext.csv')
# oof_512_folds_012_df = pd.read_csv(OOF_DIR / 'microsoft-deberta-v3-base-v2-folds_012.csv')
# oof_512_folds_34_df = pd.read_csv(OOF_DIR / 'microsoft-deberta-v3-base-v2-folds_34.csv')
# oof_512_df = pd.concat([oof_512_folds_012_df, oof_512_folds_34_df])
oof_512_df = pd.read_csv(OOF_DIR / 'allenai-longformer-base-4096-tokcls-v1.csv')

In [5]:
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_text_token_len,pos,essay_text,discourse_text_len,essay_text_len
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,84,0,"Hi, i'm Isaac, i'm going to be writing about h...",84,432
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,50,1,"Hi, i'm Isaac, i'm going to be writing about h...",50,432
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,25,2,"Hi, i'm Isaac, i'm going to be writing about h...",25,432
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,93,3,"Hi, i'm Isaac, i'm going to be writing about h...",93,432
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,23,4,"Hi, i'm Isaac, i'm going to be writing about h...",23,432
...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,25,0,Some people may ask multiple people for advice...,25,195
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,12,1,Some people may ask multiple people for advice...,12,195
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,27,2,Some people may ask multiple people for advice...,27,195
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,93,3,Some people may ask multiple people for advice...,93,195


In [6]:
def _preprocess_tensor_str(x: str | float) -> float:
    if isinstance(x, float):
        return x
    return float(x[7:-1])


oof_512_df['ineffective_score'] = oof_512_df['ineffective_score'].progress_apply(_preprocess_tensor_str)
oof_512_df['adequate_score'] = oof_512_df['adequate_score'].progress_apply(_preprocess_tensor_str)
oof_512_df['effective_score'] = oof_512_df['effective_score'].progress_apply(_preprocess_tensor_str)

  0%|          | 0/36765 [00:00<?, ?it/s]

  0%|          | 0/36765 [00:00<?, ?it/s]

  0%|          | 0/36765 [00:00<?, ?it/s]

In [7]:
oof_512_df

,id,ineffective_score,adequate_score,effective_score
0,f8c165e9a831,0.6208,0.3699,0.0092
1,1154e0d6ac9b,0.0449,0.6912,0.2639
2,e4d2bd0ffb3c,0.3484,0.6153,0.0363
3,abec8175fd6a,0.1769,0.8049,0.0182
4,8d99807b6030,0.0752,0.8503,0.0745
...,...,...,...,...
36760,9a71c37e4b7a,0.0281,0.4826,0.4893
36761,48b4b78d6c2c,0.3428,0.6481,0.0091
36762,1490dd287222,0.0090,0.2484,0.7426
36763,e1db3779bb5a,0.0723,0.7804,0.1473


In [8]:
def _get_error(row: t.Dict[str, t.Any]) -> float:
    eff = row['discourse_effectiveness']
    if eff == 'Ineffective':
        return 1.0 - row['ineffective_score']
    if eff == 'Adequate':
        return 1.0 - row['adequate_score']
    if eff == 'Effective':
        return 1.0 - row['effective_score']
    raise NotImplementedError()


def _add_error_col(df: pd.DataFrame, oof_df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.merge(oof_df, left_on='discourse_id', right_on='id')
    df['error'] = df.progress_apply(_get_error, axis=1)
    return df

train_df = _add_error_col(train_df, oof_512_df)

  0%|          | 0/36765 [00:00<?, ?it/s]

In [9]:
train_df

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_text_token_len,pos,essay_text,discourse_text_len,essay_text_len,id,ineffective_score,adequate_score,effective_score,error
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,84,0,"Hi, i'm Isaac, i'm going to be writing about h...",84,432,0013cc385424,0.0893,0.8104,0.1003,0.1896
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,50,1,"Hi, i'm Isaac, i'm going to be writing about h...",50,432,9704a709b505,0.0933,0.8667,0.0400,0.1333
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,25,2,"Hi, i'm Isaac, i'm going to be writing about h...",25,432,c22adee811b6,0.1025,0.8636,0.0339,0.1364
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,93,3,"Hi, i'm Isaac, i'm going to be writing about h...",93,432,a10d361e54e4,0.5075,0.4890,0.0035,0.5110
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,23,4,"Hi, i'm Isaac, i'm going to be writing about h...",23,432,db3e453ec4e2,0.1144,0.8613,0.0242,0.1387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,25,0,Some people may ask multiple people for advice...,25,195,9f63b687e76a,0.1175,0.8456,0.0369,0.1544
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,12,1,Some people may ask multiple people for advice...,12,195,9d5bd7d86212,0.2671,0.7236,0.0092,0.2764
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,27,2,Some people may ask multiple people for advice...,27,195,f1b78becd573,0.1650,0.7616,0.0734,0.2384
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,93,3,Some people may ask multiple people for advice...,93,195,cc184624ca8e,0.5114,0.4801,0.0085,0.4886


In [10]:
train_df.sort_values('error')[['discourse_type', 'discourse_text', 'discourse_effectiveness', 'discourse_text_len', 'essay_text_len', 'error']].iloc[:20]

,discourse_type,discourse_text,discourse_effectiveness,discourse_text_len,essay_text_len,error
19366,Claim,The Face is a natural rock formation.,Adequate,10,259,0.0401
13599,Position,The face on Mars is just a natural landform,Adequate,12,252,0.0403
35640,Position,So overall i feel like student/ kids shouldnt ...,Adequate,19,508,0.0424
9390,Position,I go against this technology to read students ...,Adequate,18,429,0.0433
10061,Position,this is just a natural landform created on Mar...,Adequate,13,503,0.0445
16828,Position,The Face on Mars is just a natural landform.\n...,Adequate,20,285,0.0447
26779,Position,This face is just a natural landform,Adequate,10,231,0.0451
26746,Claim,My last reason is becasue with the 3D high-res...,Adequate,29,225,0.0455
11823,Position,Using this technology that could read emotiona...,Adequate,15,325,0.0463
8609,Position,The Facial Action Coding System could be valua...,Adequate,12,235,0.0479


In [11]:
train_df.sort_values('error', ascending=False)[['discourse_type', 'discourse_text', 'discourse_effectiveness', 'discourse_text_len', 'essay_text_len', 'error']].iloc[:20]

,discourse_type,discourse_text,discourse_effectiveness,discourse_text_len,essay_text_len,error
21418,Rebuttal,The way we are voting now has no problems so far,Effective,13,287,0.9985
13707,Evidence,97 percent of carbon dioxide blankets venus an...,Effective,58,350,0.9977
18107,Evidence,#1 fact there is a certainty of outcome by tha...,Effective,176,436,0.9976
27784,Evidence,Us citixen's help choose the state's elector w...,Effective,38,698,0.9968
13710,Claim,Many scientist are looking for a way to get sa...,Effective,36,350,0.9968
17182,Evidence,"It happened in Hawii in 1960. Luckily, Vice Pr...",Effective,38,411,0.9967
11075,Claim,If you dont have one you have to walk which me...,Effective,30,342,0.9967
6890,Claim,could make students feel left out because thei...,Ineffective,15,850,0.9960
6888,Claim,"Also, a student's talent may not represented i...",Ineffective,22,850,0.9960
6887,Claim,"First issue is, it extends an already drawn ou...",Ineffective,24,850,0.9957


In [12]:
train_df['error'].mean()

0.40548936760505916

In [13]:
train_df[train_df['essay_text_len'] < 512]['error'].mean(), \
train_df[train_df['essay_text_len'] >= 512]['error'].mean()

(0.40300079479076895, 0.40876264248378363)

In [14]:
np.corrcoef(train_df['discourse_text_len'].to_numpy(), train_df['error'].to_numpy())[0][1], \
np.corrcoef(train_df['essay_text_len'].to_numpy(), train_df['error'].to_numpy())[0][1]

(-0.04204335534589307, -0.014064413218422521)

In [15]:
train_df.groupby('discourse_type')['error'].mean()

discourse_type
Claim                   0.391580
Concluding Statement    0.393412
Counterclaim            0.396564
Evidence                0.429372
Lead                    0.425132
Position                0.362105
Rebuttal                0.456430
Name: error, dtype: float64

In [16]:
train_df.groupby('discourse_effectiveness')['error'].mean()

discourse_effectiveness
Adequate       0.359385
Effective      0.365871
Ineffective    0.612332
Name: error, dtype: float64

In [17]:
train_df.groupby('discourse_effectiveness')['id'].count()

discourse_effectiveness
Adequate       20977
Effective       9326
Ineffective     6462
Name: id, dtype: int64

In [18]:
train_df['discourse_type'].unique()

array(['Lead', 'Position', 'Claim', 'Evidence', 'Counterclaim',
       'Rebuttal', 'Concluding Statement'], dtype=object)

In [19]:
train_df[(train_df['error'] >= 0.35) & (train_df['error'] <= 0.55)].sample(n=10)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_text_token_len,pos,essay_text,discourse_text_len,essay_text_len,id,ineffective_score,adequate_score,effective_score,error
18142,68d810e68433,32B2D5592E1D,The first I ready to talk about some expectati...,Claim,Adequate,12,1,This teens manual I talk to talk about expecta...,12,704,68d810e68433,0.5109,0.4873,0.0017,0.5127
21302,a14fae1e18ac,5D1A08FC3563,The meaning of a democracy is that the people ...,Lead,Adequate,18,0,The meaning of a democracy is that the people ...,18,533,a14fae1e18ac,0.1177,0.5870,0.2953,0.4130
14980,c503c0ed9639,07E503B34962,I think all of the kids in our community shoul...,Position,Adequate,31,0,"Dear,\n\nTEACHER_NAME\n\nI think all of the ki...",31,234,c503c0ed9639,0.0398,0.6027,0.3575,0.3973
4538,864bc6104152,50F1D8786126,The three reasons about how to ¨Make Mona Lisa...,Claim,Adequate,17,1,"In the article ¨Making Mona Lisa Smile,¨ the a...",17,399,864bc6104152,0.4837,0.5150,0.0013,0.4850
9574,0606003ec7cb,AC71A062B952,"The ""hands free"" law shouldn't be a law.",Claim,Adequate,15,3,"Today, majority of humans own and operate cell...",15,552,0606003ec7cb,0.0643,0.6175,0.3182,0.3825
18254,1ac9653b4467,3493BED78E8E,We should abolish the Electoral College and ch...,Lead,Adequate,61,0,We should abolish the Electoral College and ch...,61,684,1ac9653b4467,0.0223,0.4941,0.4836,0.5059
18078,e0b51e316e15,318F40822873,it can be an important quality for other stude...,Rebuttal,Adequate,22,16,Along with the growing influence of technology...,22,993,e0b51e316e15,0.0195,0.4542,0.5263,0.5458
22992,f405f612635d,7361AA14D6EE,""" balancing the large states will help more kn...",Evidence,Adequate,122,6,"Dear State Senator,\n\nToday I would like to p...",122,521,f405f612635d,0.3836,0.5824,0.0340,0.4176
32869,99d1a9942c8b,FF07E6BF0858,In fact it just went to help thier candidate m...,Evidence,Ineffective,110,3,"Dear Senator,\n\nAn Electoral College is a vot...",110,420,99d1a9942c8b,0.4594,0.5306,0.0099,0.5406
13418,e7652de3020f,F280DC36C974,"It tells how a person it feeling, in my oppain...",Evidence,Adequate,25,2,In this article I would like to have the techn...,25,240,e7652de3020f,0.3922,0.6064,0.0014,0.3936


In [20]:
for i in range(20):
    recall = len(train_df[(train_df['ineffective_score'] >= i / 20) & (train_df['discourse_effectiveness'] == 'Ineffective')]) / len(train_df[train_df['discourse_effectiveness'] == 'Ineffective'])
    precision = len(train_df[(train_df['ineffective_score'] >= i / 20) & (train_df['discourse_effectiveness'] == 'Ineffective')]) / len(train_df[train_df['ineffective_score'] >= i / 20])
    f1 = 2 * precision * recall / (precision + recall)
    print(f't = {i / 20:.2f}, f1 = {f1:.4f}')

t = 0.00, f1 = 0.2990
t = 0.05, f1 = 0.4151
t = 0.10, f1 = 0.4625
t = 0.15, f1 = 0.4957
t = 0.20, f1 = 0.5185
t = 0.25, f1 = 0.5278
t = 0.30, f1 = 0.5232
t = 0.35, f1 = 0.5139
t = 0.40, f1 = 0.4959
t = 0.45, f1 = 0.4682
t = 0.50, f1 = 0.4326
t = 0.55, f1 = 0.3795
t = 0.60, f1 = 0.3258
t = 0.65, f1 = 0.2534
t = 0.70, f1 = 0.1904
t = 0.75, f1 = 0.1174
t = 0.80, f1 = 0.0624
t = 0.85, f1 = 0.0229
t = 0.90, f1 = 0.0037


ZeroDivisionError: division by zero